In [1]:
import numpy as np
import scipy as sp
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
from keras import optimizers
from keras import models
from keras import layers
from keras import losses
from keras import metrics

Using TensorFlow backend.


In [3]:
import time
import os
import sys
import pickle

In [4]:
db_dir = "/home/sweet/2-coursework/725bmes/eeg_deeplearning/data_extraction/src"
ME_db_fname = "mesh_ME_db.pickle"
MI_db_fname = "prelim_MI_db.pickle"

ME_db = {}
MI_db = {}


t1 = time.time()
with open(db_dir + "/" + ME_db_fname, "rb") as f:
    ME_db = pickle.load(f)
print("Loaded ME database in %f s" % (time.time()-t1))

Loaded ME database in 15.265196 s


In [5]:
np.random.seed(0)
def create_rand_seq_permutations(db):
    seq_idx = {}
    for key in db.keys():
        if key not in seq_idx:
            seq_idx[key] = None
        seq_idx[key] = np.random.permutation(np.arange(len(db[key])))
    
    return seq_idx

seq_perms = create_rand_seq_permutations(ME_db)

In [7]:
np.random.seed(0)
def generate_batch(db, perm, batch_size=4, shuffle=False):
    idx = 0
    
    while True:
        output_seq = []
        output_idx = np.array([])
        labels = np.array([])
        for key in perm.keys():
            output_idx = np.concatenate([output_idx, perm[key][idx:idx+batch_size]])
            labels = np.concatenate([labels, key*np.ones(batch_size, dtype='int')])
        if shuffle:
            rand_idx = np.random.permutation(np.arange(len(output_idx)))
            output_idx = output_idx[rand_idx]
            labels = labels[rand_idx]
        
        output_idx = output_idx.astype('int')
        labels = labels.astype('int')
        
        for i in range(len(labels)):
            output_seq.append(db[labels[i]][output_idx[i]])
        
        yield output_seq, labels
        idx += batch_size
                          
# batch_gen = generate_batch(ME_db, seq_perms, 4)
# output_seq, labels = next(batch_gen)
# print(output_seq[0][0].shape)
# j = 0
# for output_seq, labels in batch_gen:
#     if len(labels) == 0:
#         break
#     print(labels)
#     j += 1
# print(j)